## Preprocessing

In [1]:
# Import our dependencies
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
import pandas as pd
import tensorflow as tf

#  Import and read the diabetes_data.csv.
diabtetes_df = pd.read_csv("Resources/diabetes_data.csv")
diabtetes_df.head()

,Diabetes_012,HighBP,HighChol,CholCheck,BMI,Smoker,Stroke,HeartDiseaseorAttack,PhysActivity,Fruits,...,AnyHealthcare,NoDocbcCost,GenHlth,MentHlth,PhysHlth,DiffWalk,Sex,Age,Education,Income
0,0.0,1.0,1.0,1.0,40.0,1.0,0.0,0.0,0.0,0.0,...,1.0,0.0,5.0,18.0,15.0,1.0,0.0,9.0,4.0,3.0
1,0.0,0.0,0.0,0.0,25.0,1.0,0.0,0.0,1.0,0.0,...,0.0,1.0,3.0,0.0,0.0,0.0,0.0,7.0,6.0,1.0
2,0.0,1.0,1.0,1.0,28.0,0.0,0.0,0.0,0.0,1.0,...,1.0,1.0,5.0,30.0,30.0,1.0,0.0,9.0,4.0,8.0
3,0.0,1.0,0.0,1.0,27.0,0.0,0.0,0.0,1.0,1.0,...,1.0,0.0,2.0,0.0,0.0,0.0,0.0,11.0,3.0,6.0
4,0.0,1.0,1.0,1.0,24.0,0.0,0.0,0.0,1.0,1.0,...,1.0,0.0,2.0,3.0,0.0,0.0,0.0,11.0,5.0,4.0


In [2]:
# Determine the number of unique values in each column.
diabtetes_df.nunique()

Diabetes_012             3
HighBP                   2
HighChol                 2
CholCheck                2
BMI                     84
Smoker                   2
Stroke                   2
HeartDiseaseorAttack     2
PhysActivity             2
Fruits                   2
Veggies                  2
HvyAlcoholConsump        2
AnyHealthcare            2
NoDocbcCost              2
GenHlth                  5
MentHlth                31
PhysHlth                31
DiffWalk                 2
Sex                      2
Age                     13
Education                6
Income                   8
dtype: int64

In [3]:
# Split our preprocessed data into our features and target arrays
y = diabtetes_df['Diabetes_012'].values
X = diabtetes_df.drop('Diabetes_012', axis=1).values

# Split the preprocessed data into a training and testing dataset
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=42)

In [4]:
# Create a StandardScaler instances
scaler = StandardScaler()

# Fit the StandardScaler
X_scaler = scaler.fit(X_train)

# Scale the data
X_train_scaled = X_scaler.transform(X_train)
X_test_scaled = X_scaler.transform(X_test)

## Compile, Train and Evaluate the Model

In [5]:
from tensorflow.keras import regularizers

# Define a more complex neural network model with L2 regularization
model = tf.keras.models.Sequential()

# First hidden layer with L2 regularization
model.add(tf.keras.layers.Dense(units=32, activation='relu', input_dim=X_train_scaled.shape[1],
                                kernel_regularizer=regularizers.l2(0.01)))

# Second hidden layer with L2 regularization
model.add(tf.keras.layers.Dense(units=16, activation='relu', 
                                kernel_regularizer=regularizers.l2(0.01)))

# Third hidden layer with L2 regularization
model.add(tf.keras.layers.Dense(units=8, activation='relu', 
                                kernel_regularizer=regularizers.l2(0.01)))

# Add dropout for additional regularization
model.add(tf.keras.layers.Dropout(0.3))

# Output layer
model.add(tf.keras.layers.Dense(units=3, activation='softmax'))

# Check the structure of the model
model.summary()


C:\Users\Victor Doan\anaconda3\Lib\site-packages\keras\src\layers\core\dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ dense (Dense)                        │ (None, 32)                  │             704 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_1 (Dense)                      │ (None, 16)                  │             528 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_2 (Dense)                      │ (None, 8)                   │             136 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dropout (Dropout)                    │ (None, 8)                   │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_3 (Dense)                      │ (None, 3)                   │              27 │
└──────────────────────────────────────┴─────────────────────────────┴─────────────────┘

 Total params: 1,395 (5.45 KB)

 Trainable params: 1,395 (5.45 KB)

 Non-trainable params: 0 (0.00 B)

In [7]:
# Compile the model
model.compile(loss="sparse_categorical_crossentropy", optimizer="adam", metrics=["accuracy"])


In [8]:
# Train the model
fit_model = model.fit(X_train_scaled, y_train, epochs=100)


Epoch 1/100
5946/5946 ━━━━━━━━━━━━━━━━━━━━ 6s 725us/step - accuracy: 0.8194 - loss: 0.6455
Epoch 2/100
5946/5946 ━━━━━━━━━━━━━━━━━━━━ 4s 732us/step - accuracy: 0.8406 - loss: 0.4335
Epoch 3/100
5946/5946 ━━━━━━━━━━━━━━━━━━━━ 4s 714us/step - accuracy: 0.8432 - loss: 0.4275
Epoch 4/100
5946/5946 ━━━━━━━━━━━━━━━━━━━━ 4s 716us/step - accuracy: 0.8461 - loss: 0.4236
Epoch 5/100
5946/5946 ━━━━━━━━━━━━━━━━━━━━ 4s 716us/step - accuracy: 0.8445 - loss: 0.4225
Epoch 6/100
5946/5946 ━━━━━━━━━━━━━━━━━━━━ 4s 717us/step - accuracy: 0.8449 - loss: 0.4209
Epoch 7/100
5946/5946 ━━━━━━━━━━━━━━━━━━━━ 4s 718us/step - accuracy: 0.8444 - loss: 0.4220
Epoch 8/100
5946/5946 ━━━━━━━━━━━━━━━━━━━━ 4s 712us/step - accuracy: 0.8451 - loss: 0.4199
Epoch 9/100
5946/5946 ━━━━━━━━━━━━━━━━━━━━ 4s 708us/step - accuracy: 0.8442 - loss: 0.4223
Epoch 10/100
5946/5946 ━━━━━━━━━━━━━━━━━━━━ 4s 709us/step - accuracy: 0.8437 - loss: 0.4194
Epoch 11/100
5946/5946 ━━━━━━━━━━━━━━━━━━━━ 4s 715us/step - accuracy: 0.8426 - loss: 0.41

In [9]:
# Evaluate the model using the test data
model_loss, model_accuracy = nn.evaluate(X_test_scaled,y_test,verbose=2)
print(f"Loss: {model_loss}, Accuracy: {model_accuracy}")

1982/1982 - 1s - 735us/step - accuracy: 0.8487 - loss: 0.3922
Loss: 0.39223772287368774, Accuracy: 0.8487070202827454


In [10]:
# Export our model to HDF5 file
model.save('diabetes_prediction_model_r1.h5')